In [1]:
import json
import os
import os.path as osp
import sys
from PIL import Image
import pickle as pkl
import cv2

from detectron.datasets.json_dataset import JsonDataset
from detectron.utils.io import load_object
import detectron.utils.vis as vis_utils


visPath = '/home/chnxi/Detectron/backfill/visualizations/HF_Detector/'
cls_thrsh_file = '/home/chnxi/Detectron/backfill/HF_Detector/class_thresholds_at_pr.pkl'
cls_thrsh_dict = load_object(cls_thrsh_file)
dataset = 'FashionV2_train' # {'HF', 'OpenImage'}
#bboxFile = '/home/chnxi/ssd/backfill/FashionV2_on_OpenImage/outputs/FashionOnOIVal_VOC.json
#imgPath = '/media/data/chnxi/OpenImage/validation_images/'
if dataset == 'FashionV2_train':
    bboxFile = '/home/chnxi/Detectron/backfill/HF_Detector/FashionV2_train/test/FashionV2_train/retinanet/detections.pkl' #sys.argv[1]
    imgPath = '/media/data/chnxi/FashionV2/Images/'
elif dataset == 'OpenImage':
    #bboxFile = '/home/chnxi/ssd/backfill/FashionV2_on_OpenImage/outputs/_FashionV2_OpenImageTrain.json' #sys.argv[1]
    bboxFile = '/home/chnxi/Detectron/backfill/HF_Detector/OpenImage_train/test/OpenImage_train/retinanet/detections.pkl' #sys.argv[1]
    imgPath = '/media/data/chnxi/OpenImage/train_images/'
elif dataset == 'coco_2014_valminusminival':
    bboxFile = '/home/chnxi//Detectron/backfill/HF_Detector/coco_trainval/test/coco_2014_valminusminival/retinanet/detections.pkl' #sys.argv[1]
    imgPath = '/media/data/chnxi/coco/val2014/'
    
visFolder = osp.join(visPath, dataset)
if not osp.isdir(visFolder):
    os.makedirs(visFolder)
    
ds = JsonDataset(dataset)
roidb = ds.get_roidb()
    
print("loading {}".format(bboxFile))
dets = load_object(bboxFile)
print ("loaded")
all_boxes = dets['all_boxes']


================= loading annotation FashionV2_train
loading annotations into memory...
Done (t=3.99s)
creating index...
index created!
getting roidb
loading /home/chnxi/Detectron/backfill/HF_Detector/FashionV2_train/test/FashionV2_train/retinanet/detections.pkl
loaded


In [ ]:
cls_thrsh_file = '/home/chnxi/Detectron/backfill/HF_Detector/class_thresholds_at_pr.pkl'
cls_thrsh_dict = load_object(cls_thrsh_file)
print (cls_thrsh_dict.keys())
imgNum = len(roidb)
classes = cls_thrsh_dict['classes']

print(len(cls_thrsh_dict['prec_at'][0.9]))
print (len(classes))

print(roidb[0])

In [3]:
# cls_prthrsh_dict = {}
# for cid, cls in enumerate(classes):
#     cls_thrshs = {'prec_at_0.9': cls_thrsh_dict['prec_at'][0.9][cid], 
#                   'prec_at_0.8': cls_thrsh_dict['prec_at'][0.8][cid],
#                   'rec_at_0.5' : cls_thrsh_dict['rec_at'][0.5][cid],
#                   'rec_at_0.7' : cls_thrsh_dict['rec_at'][0.7][cid]}
#     cls_prthrsh_dict[cid] = cls_thrshs

new_cls_thrsh_dict = {'prec_at_0.9': cls_thrsh_dict['prec_at'][0.9], 
                  'prec_at_0.8': cls_thrsh_dict['prec_at'][0.8],
                  'rec_at_0.5' : cls_thrsh_dict['rec_at'][0.5],
                  'rec_at_0.7' : cls_thrsh_dict['rec_at'][0.7]}
    
print (new_cls_thrsh_dict['prec_at_0.9'])

[0.65890813 0.40394038 0.5774321  0.77713573 0.60711682 0.67322326
 0.52851528 0.46174601 0.61438024 0.64767092 0.81435126 0.53951627
 0.57961816 0.60969418 0.62125641 0.5989415  0.58761984 0.63688034
 0.59885871 0.5786556  0.66398668 0.59098428 0.64770222 0.60134423
 0.71601284 0.78756082 0.59866714 0.53077555 0.55953896 0.68613309
 0.84925234 0.65713745 0.65950334 0.56061822 0.70493436 0.56271166
 0.70769304 0.77569616 0.63649017 0.65164274 0.57107413 0.57096714
 0.59916556 0.6672796  0.57261157 0.66317564 0.66976112 0.59649551
 0.57466114 0.69485384 0.66893601 0.78391629 0.78418601 0.35948369
 0.55339015 0.76437533 0.76778102 0.50362223]


In [ ]:
# print (new_cls_thrsh_dict['prec_at_0.9'])
import os
import os.path as osp
visPath = '../visualizations/'
if not osp.isdir(visPath):
    os.makedirs(visPath)
    
keylist = ['prec_at_0.9', 'prec_at_0.8', 'rec_at_0.5', 'rec_at_0.7']
color_list = ['g', 'r', 'b', 'm']

def visImageClsThrshs(plt, splot, im, imgBoxes, ecolor='m', title='', showCateg=False):
    #plt.cla()
    fx, fy, fi = splot
    plt.subplot(fx, fy, fi)
    plt.imshow(im)
    plt.axis('off')
    plt.title(title)
    ax = plt.gca()
    for cid, cls_boxes in enumerate(imgBoxes):
        for obj in cls_boxes:
            #print obj
            bbox = obj[:4]
            score = obj[-1]
            categ = classes[cid]            
            #ecolor = 'm' #if gid == jmax else gtcolor
            #print "{}: bbox = {}".format(categ, bbox)
            ax.add_patch(
                plt.Rectangle((bbox[0], bbox[1]),
                              bbox[2] - bbox[0],
                              bbox[3] - bbox[1], fill=False,
                              edgecolor=ecolor, linestyle='-', linewidth=0.5))
            if showCateg:
                ax.text(bbox[0], bbox[3], '{}:{:.1f}'.format(categ, score), bbox={'facecolor': ecolor, 'alpha': 0.5})
        plt.draw()

import matplotlib.pyplot as plt
from PIL import Image
saveVis = False
showVis = True
        
import random
visNum = 100
imgIds = random.sample(range(imgNum), visNum)
print(imgIds)
    
def id_or_index(ix, val):
    if len(val) == 0:
        return val
    else:
        return val[ix]
        
for ix, entry in enumerate(roidb):
    imgFileName = entry['image']
    imgBaseName = osp.basename(imgFileName)
    print (imgBaseName)
    print (imgFileName)
    im = Image.open(imgFileName)
    img_boxes = [
        id_or_index(ix, cls_k_boxes) for cls_k_boxes in all_boxes
    ]
    img_boxes = img_boxes[1:] # remove background boxes
        
    f, axarr = plt.subplots(1, 4, sharey=True, figsize=(20, 20))
    plt.cla()
    fid = 0
    total_box_num = 0
    for fid, key in enumerate(keylist): #['prec_at_0.9']: #new_cls_thrsh_dict:
        print (key)
        ax = axarr[fid]
        thresholds = new_cls_thrsh_dict[key]
        img_boxes_thrshed_num = 0
        img_boxes_thrshed = []
        for cid, cls_boxes in enumerate(img_boxes):
            #print (cls_boxes.shape)
            cls_boxes_thrshed = [bbox for bbox in cls_boxes if bbox[-1] > thresholds[cid]]
            cls_boxes_num = len(cls_boxes_thrshed)
            img_boxes_thrshed.append(cls_boxes_thrshed)
            img_boxes_thrshed_num += cls_boxes_num
        print(img_boxes_thrshed_num)
        total_box_num += img_boxes_thrshed_num
        visImageClsThrshs(plt, (1,4, fid+1), im, img_boxes_thrshed, ecolor=color_list[fid], title=key, showCateg=True)
        fid += 1
    #plt.show()
    if total_box_num > 0:
        plt.savefig(osp.join(visFolder,imgBaseName), bbox_inches='tight',pad_inches = 0)
    plt.close()
print ("Done!")

[30609, 16758, 699, 23019, 21107, 13423, 30046, 2091, 29500, 35302, 12140, 4966, 9280, 19948, 26118, 22587, 31938, 22749, 12737, 18263, 18028, 9142, 22472, 33268, 35474, 2769, 19024, 26916, 33308, 18067, 26697, 5631, 383, 26749, 30066, 17029, 24185, 24052, 30793, 32556, 6703, 24489, 28840, 26280, 12978, 7349, 14737, 24328, 4922, 8965, 5918, 35503, 18922, 24729, 5189, 23810, 33898, 32481, 10501, 34997, 6899, 12754, 14362, 30653, 28787, 15562, 31553, 3702, 14543, 8571, 15224, 32987, 9514, 7515, 4005, 13289, 26590, 19109, 17345, 25414, 33579, 3970, 16766, 30221, 12311, 32494, 21439, 264, 4634, 17853, 20055, 10736, 29020, 32602, 4895, 20007, 18505, 20254, 34654, 19858]
COCO_val2014_000000000042.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000000042.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
2
rec_at_0.7
33
COCO_val2014_000000000073.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000

prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
COCO_val2014_000000000623.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000000623.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
2
COCO_val2014_000000000626.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000000626.jpg
prec_at_0.9
0
prec_at_0.8
1
rec_at_0.5
2
rec_at_0.7
9
COCO_val2014_000000000636.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000000636.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
5
rec_at_0.7
22
COCO_val2014_000000000641.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000000641.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
1
rec_at_0.7
8
COCO_val2014_000000000661.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000000661.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5

prec_at_0.8
0
rec_at_0.5
3
rec_at_0.7
8
COCO_val2014_000000001176.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001176.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
4
rec_at_0.7
20
COCO_val2014_000000001180.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001180.jpg
prec_at_0.9
1
prec_at_0.8
1
rec_at_0.5
17
rec_at_0.7
58
COCO_val2014_000000001205.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001205.jpg
prec_at_0.9
0
prec_at_0.8
2
rec_at_0.5
15
rec_at_0.7
52
COCO_val2014_000000001228.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001228.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
16
COCO_val2014_000000001244.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001244.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_a

prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
6
COCO_val2014_000000001803.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001803.jpg
prec_at_0.9
0
prec_at_0.8
1
rec_at_0.5
13
rec_at_0.7
27
COCO_val2014_000000001840.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001840.jpg
prec_at_0.9
0
prec_at_0.8
1
rec_at_0.5
25
rec_at_0.7
73
COCO_val2014_000000001841.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001841.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
COCO_val2014_000000001852.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001852.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
2
rec_at_0.7
28
COCO_val2014_000000001856.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000001856.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at

prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
COCO_val2014_000000002621.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000002621.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
8
COCO_val2014_000000002640.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000002640.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
1
rec_at_0.7
16
COCO_val2014_000000002684.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000002684.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
21
COCO_val2014_000000002690.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000002690.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
3
rec_at_0.7
18
COCO_val2014_000000002753.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000002753.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0

...

prec_at_0.8
0
rec_at_0.5
2
rec_at_0.7
16
COCO_val2014_000000066567.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000066567.jpg
prec_at_0.9
1
prec_at_0.8
6
rec_at_0.5
19
rec_at_0.7
37
COCO_val2014_000000066568.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000066568.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
14
rec_at_0.7
82
COCO_val2014_000000066632.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000066632.jpg
prec_at_0.9
1
prec_at_0.8
2
rec_at_0.5
20
rec_at_0.7
67
COCO_val2014_000000066650.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000066650.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
4
rec_at_0.7
13
COCO_val2014_000000066675.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000066675.jpg
prec_at_0.9
0
prec_at_0.8
1
rec

prec_at_0.8
0
rec_at_0.5
7
rec_at_0.7
41
COCO_val2014_000000067419.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067419.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
7
COCO_val2014_000000067422.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067422.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
COCO_val2014_000000067427.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067427.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
3
rec_at_0.7
7
COCO_val2014_000000067431.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067431.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
5
rec_at_0.7
27
COCO_val2014_000000067455.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067455.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.

prec_at_0.8
0
rec_at_0.5
1
rec_at_0.7
7
COCO_val2014_000000067969.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067969.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
1
rec_at_0.7
7
COCO_val2014_000000067975.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067975.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
COCO_val2014_000000067994.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067994.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
5
rec_at_0.7
23
COCO_val2014_000000067995.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000067995.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
7
COCO_val2014_000000068005.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000068005.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5

prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
9
COCO_val2014_000000068546.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000068546.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
0
COCO_val2014_000000068646.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000068646.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
6
rec_at_0.7
25
COCO_val2014_000000068650.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000068650.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
2
COCO_val2014_000000068674.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000068674.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
1
rec_at_0.7
3
COCO_val2014_000000068715.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000068715.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5

prec_at_0.8
0
rec_at_0.5
2
rec_at_0.7
16
COCO_val2014_000000069512.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000069512.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
2
rec_at_0.7
14
COCO_val2014_000000069535.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000069535.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
4
rec_at_0.7
17
COCO_val2014_000000069536.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000069536.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
4
rec_at_0.7
20
COCO_val2014_000000069544.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000069544.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_0.5
0
rec_at_0.7
1
COCO_val2014_000000069553.jpg
/media/data/chnxi/github/Detectron_20190306/detectron/datasets/data/coco/val2014/COCO_val2014_000000069553.jpg
prec_at_0.9
0
prec_at_0.8
0
rec_at_